In [4]:
# now we will test our model on test data
# first we train our model on train data without any preprocessing

# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import cv2
import pydicom
from keras.models import Model
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.utils import to_categorical
from sklearn.metrics import confusion_matrix, classification_report
from keras.applications.densenet import DenseNet121
#from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.inception_v3 import InceptionV3
from keras.applications.nasnet import NASNetMobile



# load data
train = pd.read_csv('train_calc_annotations.csv')
test = pd.read_csv('test_calc_annotations.csv')

# create train and test data
train_data = []
test_data = []

for i in range(len(train)):
    # load dicom images
    image = pydicom.dcmread(train['cropped_file'][i]).pixel_array
    # resize images
    image = cv2.resize(image, (224, 224))
    # normalize images
    image = image / 255.0
    # append data
    train_data.append(image)

for i in range(len(test)):
    # load dicom images
    image = pydicom.dcmread(test['cropped_file'][i]).pixel_array
    # resize images
    image = cv2.resize(image, (224, 224))
    # normalize images
    image = image / 255.0
    # append data
    test_data.append(image)

# convert data into numpy array
train_data = np.array(train_data)
test_data = np.array(test_data)

# create train and test labels
train_labels = train['pathology'].values
test_labels = test['pathology'].values

# convert labels into categorical
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

# create model
base_model = DenseNet121(include_top=False, weights='imagenet', input_shape=(224, 224, 3))
x = base_model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
predictions = Dense(2, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=predictions)

# compile model
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

# train model
history = model.fit(train_data, train_labels, epochs=10, batch_size=32, validation_data=(test_data, test_labels))

# save model
model.save('model.h5')

# save history
with open('history.pkl', 'wb') as file:
    pickle.dump(history.history, file)

# plot accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend(['Train', 'Test'])
plt.show()

# plot loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend(['Train', 'Test'])
plt.show()

# predict test data
predictions = model.predict(test_data)
predictions = np.argmax(predictions, axis=1)
test_labels = np.argmax(test_labels, axis=1)

# create confusion matrix
cm = confusion_matrix(test_labels, predictions)
sns.heatmap(cm, annot=True)
plt.show()

# create classification report
print(classification_report(test_labels, predictions))


ModuleNotFoundError: No module named 'tensorflow'